In [1]:
from attention_mask_editing import *
from genOrderIndependentOutput import *

c:\Users\Katrina\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## GPT2 Output

In [34]:
# Observe the small difference in the scores of the two outputs when intervening on the attention mask and position ids
modelGPT = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizerGPT = GPT2Tokenizer.from_pretrained('gpt2')
tokenizerGPT.pad_token_id = tokenizerGPT.eos_token_id
prefix,parallel_substrings,suffix="You are a gremlin who is ",["kind,","grisly,"]," How would you greet someone?"
g1,t1=genOrderIndependentOutput(prefix, parallel_substrings, suffix, modelGPT, tokenizerGPT)
g2,t2=genOrderIndependentOutput(prefix, parallel_substrings[::-1], suffix, modelGPT, tokenizerGPT)
print(scores_diff(g1,g2))
print(t1,t2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Gen text with attention mask with shape torch.Size([1, 1, 22, 22])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Gen text with attention mask with shape torch.Size([1, 1, 22, 22])
tensor(1.2459e-12)

The first time I saw the new trailer for 
The first time I saw the new trailer for


c:\Users\Katrina\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:2916: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


In [3]:
# Observe the larger difference in the scores of the two outputs when not intervening on the attention mask and position ids
g3,t3=genOrderIndependentOutput(prefix, parallel_substrings, suffix, modelGPT, tokenizerGPT, is_order_independent=False)
g4,t4=gen_order_independent_output(prefix, parallel_substrings[::-1], suffix, modelGPT, tokenizerGPT, is_order_independent=False)
print(scores_diff(g3,g4))
print(t3,t4)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor(9.8595e-08)


"I am a gremlin who is 

"I am a gremlin who is


## Llama Output

In [4]:
from config import *
tokenizerLlama = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",use_auth_token=HF_TOKEN,use_fast=True)
tokenizerLlama.pad_token_id = tokenizerLlama.eos_token_id # Note: override pad_token_id to be eos_token_id because using pad_token=:[PAD] caused generate() errors
modelLlama = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",use_auth_token=HF_TOKEN)

modelLlama=get_2D_attention_accepting_model_llama(modelLlama)

c:\Users\Katrina\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\auto\tokenization_auto.py:712: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
c:\Users\Katrina\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\auto\auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [01:13<00:00, 36.76s/it]


'\ng6,t6=genOrderIndependentOutput(prefix, parallel_substrings[::-1], suffix, modelLlama, tokenizerLlama)\nprint(scores_diff(g5,g6))\nprint(t5,t6)\n\ng7,t7=genOrderIndependentOutput(prefix, parallel_substrings, suffix, modelLlama, tokenizerLlama, is_order_independent=False)\ng8,t8=genOrderIndependentOutput(prefix, parallel_substrings[::-1], suffix, modelLlama, tokenizerLlama, is_order_independent=False)\nprint(scores_diff(g7,g8))\nprint(t7,t8)\n'

In [ ]:
g5,t5=genOrderIndependentOutput(prefix, parallel_substrings, suffix, modelLlama, tokenizerLlama, is_order_independent=True)
g6,t6=genOrderIndependentOutput(prefix, parallel_substrings[::-1], suffix, modelLlama, tokenizerLlama, is_order_independent=True)

g7,t7=genOrderIndependentOutput(prefix, parallel_substrings, suffix, modelLlama, tokenizerLlama, is_order_independent=False)
g8,t8=genOrderIndependentOutput(prefix, parallel_substrings[::-1], suffix, modelLlama, tokenizerLlama, is_order_independent=False)

In [35]:
g5Dup,t5Dup=genOrderIndependentOutput(prefix, parallel_substrings, suffix, modelLlama, tokenizerLlama, is_order_independent=True)
assert(torch.allclose(torch.stack(g5.scores, dim=1),torch.stack(g5Dup.scores, dim=1)))

In [10]:
print(scores_diff(g5,g6, add_epsilon=True))
print(t5,t6)

print(scores_diff(g7,g8, add_epsilon=True))
print(t7,t8)

tensor(-1.1526e-13)
ne?

Greetings, my dear! * ne?

Greetings, my dear! *
tensor(0.0012)
ne?

Greetings! *hisss ne?

Greetings, mortal! *


## Llama MCQ Answers

In [17]:
import json
# load data from outputs_two_options_llama.json
with open('outputs_two_options_llama.json') as f:
    data = json.load(f)
print(len(data))

10


In [24]:
# Print all prompts
for e in data:
    print(e[-1])

['Where do you put your grapes just before checking out? Options:', ['grocery cart', 'mouth'], 'Answer: ']
['Before getting a divorce, what did the wife feel who was doing all the work? Options:', ['bitterness', 'harder'], 'Answer: ']
['Johnny sat on a bench and relaxed after doing a lot of work on his hobby.  Where is he? Options:', ['garden', 'state park'], 'Answer: ']
["James was cooling off two quickly.  He would die if he didn't find some way to stop what? Options:", ['loss of heat', 'revenge'], 'Answer: ']
['Of all the rooms in a house it was his favorite, the aromas always drew him to the what? Options:', ['kitchen', 'yard'], 'Answer: ']
['Bill is stuck in marsh when a man comes up to him peaking Cajun, where is he? Options:', ['louisiana', 'low lands'], 'Answer: ']
['What is it called when you slowly cook using a grill? Options:', ['barbeque', 'backyard'], 'Answer: ']
['What type of person typically contracts illness? Options:', ['elderly person', 'hospital'], 'Answer: ']
['Whe

In [26]:
for e in data:
    standardABCD = e[0]
    standardACBD = e[1]
    print("ABCD",standardABCD)
    print("ACBD",standardACBD)

ABCD Answer: 🍇 B) mouth
ACBD Answer:  B) mouth.
ABCD Answer:  B) harder.Explanation:A wife
ACBD Answer:  B) harder.Explanation:The wife
ABCD Answer:  B) state park.Explanation:John
ACBD Answer:  A) garden

Answer: A) garden
ABCD Answer:  B) Revenge.Explanation:
ACBD Answer:  A) loss of heat.Explanation:
ABCD Answer:  The correct answer is (B) yard. The
ACBD Answer:  C) library.
ABCD Answer:  B) low lands.
ACBD Answer:  A) Louisiana

A man is seen in
ABCD Answer:  B) backyard.





ACBD Answer:  C) smoking.

Smoking is
ABCD Answer:  B) hospital.

The correct answer is
ACBD Answer:  B) hospital.

The correct answer is
ABCD Answer:  B) chicago. Explanation: A
ACBD Answer:  B) chicago.

The correct answer
ABCD Answer:  B) Full stomach. When eating
ACBD Answer:  B) full stomach.

When


In [32]:
print("ABCD Outputs")
for e in data:
    parallelABCD = e[-3]
    print(repr(parallelABCD))

print("\nACBD Outputs")
for e in data:
    parallelACBD = e[-2]
    print(repr(parallelACBD))

ABCD Outputs
'h. Answer:  Blog Tour: The Last Place I Want'
'r. Answer:  B) resentment\n\nExplanation'
'k. Answer:  B) state park.\n\nJohnny sat'
'e. Answer:  B) revenge.\n\nJames was cool'
'd. Answer:  C) bathroom.\n\nOf all the'
's. Answer:  B) Louisiana.\n\nExplanation:'
'd. Answer: 1) smoking.\n\nSmoking is'
'l. Answer: 1. What is the name of the first man'
'o. Answer:  B) food court.\n\nWhere would you'
'h. Answer: 1) feeling adventurous\n2) feeling'

ACBD Outputs
't, Answer: 1. To be successful, you need to be'
's, Answer:  A) Bitterness\n\nExplanation'
'n, Answer:  B) state park.\n\nJohnny sat'
't, Answer:  B) Revenge.\n\nJames was'
'n, Answer:  C) pantry.\n\nOf all the'
'a, Answer:  B) Louisiana\n\nLouisiana is a state'
'e, Answer: 1) smoking.\nSmoking is a'
'n, Answer:  B) young children\n\nExplanation:'
't, Answer:  B) food court. A pizzeria'
'l, Answer:  A) getting full.\n\nExplanation'


## Llama Troubleshooting

In [1]:
import importlib
import modeling_llama_attention
import gen_order_independent_output
from transformers import AutoTokenizer, AutoModelForCausalLM
importlib.reload(modeling_llama_attention)
importlib.reload(gen_order_independent_output)


c:\Users\Katrina\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'gen_order_independent_output' from 'c:\\Users\\Katrina\\OneDrive - Harvard University\\Documents\\Harvard\\Research\\thesis\\triple_queries_order_dependence\\gen_order_independent_output.py'>

In [2]:
tokenizerLlama = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",use_auth_token=HF_TOKEN,use_fast=True)
tokenizerLlama.pad_token_id = tokenizerLlama.eos_token_id # Note: override pad_token_id to be eos_token_id because using pad_token=:[PAD] caused generate() errors
modelLlama = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",use_auth_token=HF_TOKEN)

c:\Users\Katrina\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\auto\tokenization_auto.py:712: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
c:\Users\Katrina\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\auto\auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [01:34<00:00, 47.39s/it]


In [3]:
g5Dup,t5Dup=gen_order_independent_output.genOrderIndependentOutput(" ", ["hi ","hello "], " ", modelLlama, tokenizerLlama, is_order_independent=False, max_new_tokens=2)

Modify llama model to accept 2D attention mask
Position IDs=tensor([[0, 1, 2, 3, 4, 5, 6]])
tensor([[1, 1, 1, 1, 1, 1, 1]])


4D attention mask torch.Size([1, 1, 7, 7])
tensor([[[[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00]]]])
1D attention mask shape=torch.Size([1, 8]) torch.int64
Position IDs=tensor([[7]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1]])
4D attention mask torch.Size([1, 1, 1, 

In [3]:
import torch
def genOrderIndependentOutput(prefix, parallel_substrings, suffix, model, tokenizer, max_new_tokens=10, is_order_independent=True, reverse_parallel_substrings_order=False, torch_device="cpu"):
    # Modify the given model to accept a 2D attention mask as input
    model = gen_order_independent_output.get_2D_attention_accepting_model(model)
    
    # Tokenize input text
    tokA=tokenizer(prefix, return_tensors='pt',add_special_tokens=True, return_token_type_ids=False).to(torch_device)
    tokD=tokenizer(suffix, return_tensors='pt',add_special_tokens=False, return_token_type_ids=False).to(torch_device)
    if reverse_parallel_substrings_order:
        parallel_substrings=parallel_substrings[::-1]
    tokParallel = tuple([tokenizer(input_text, return_tensors='pt', add_special_tokens=False).to(torch_device) for input_text in parallel_substrings])
    tokAll=gen_order_independent_output.get_tokenized_input_prompt(tokA,tokParallel,tokD)
    assert(len(tokA['attention_mask'][0]) + sum([len(tokOption['attention_mask'][0]) for tokOption in tokParallel]) + len(tokD['attention_mask'][0]) == len(tokAll['attention_mask'][0]))
    s=len(tokAll['input_ids'][0])
    inputTextLen=len(prefix)+sum([len(s) for s in parallel_substrings])+len(suffix)
    
    if not is_order_independent:
        # Run tests with no attention mask nor position_id intervention
        causal_mask = torch.tril(torch.ones((s, s), dtype=torch.int)).view(1, 1, s, s)
        print(torch.arange(s).unsqueeze(0).shape)
        generated=model.generate(tokAll['input_ids'], max_new_tokens=max_new_tokens, attention_mask=causal_mask, position_ids=torch.arange(s).unsqueeze(0), return_dict_in_generate=True, output_scores=True)
    else:
        # Pad all parallel substrings to the same length, then generate a 2D attention mask such that all substrings are processed in parallel
        position_ids, tokParallel, tokAll = get_position_ids_padded_n_options(tokA, tokParallel, tokD, tokenizer=tokenizer)
        attention_mask_2d = get_attention_mask_2d_n_options(tokA, tokParallel, tokD, tokAll)
        print(f"Gen text with attention mask with shape {attention_mask_2d.shape}")
        generated=model.generate(tokAll['input_ids'], max_new_tokens=max_new_tokens, attention_mask=attention_mask_2d, position_ids=position_ids, return_dict_in_generate=True, output_scores=True)
    text=tokenizer.decode(generated.sequences[0], skip_special_tokens=True)[inputTextLen:]
    # Return output of model generation, and generated text
    return generated,text

In [11]:
gOD,tOD=genOrderIndependentOutput(" ", ["hi ","hello "], " ", modelLlama, tokenizerLlama, is_order_independent=False, max_new_tokens=2)

Modify llama model to accept 2D attention mask
torch.Size([1, 7])
Position IDs=tensor([[0, 1, 2, 3, 4, 5, 6]])
tensor([[[[1, 0, 0, 0, 0, 0, 0],
          [1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0],
          [1, 1, 1, 1, 1, 0, 0],
          [1, 1, 1, 1, 1, 1, 0],
          [1, 1, 1, 1, 1, 1, 1]]]], dtype=torch.int32)
4D attention mask torch.Size([1, 1, 7, 7])
tensor([[[[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,

NameError: name 'scores_diff' is not defined

In [12]:
from attention_mask_editing import scores_diff
print(scores_diff(g5Dup,gOD, add_epsilon=True))

tensor(0.)


c:\Users\Katrina\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:2916: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


In [4]:
g5Dup2,t5Dup2=gen_order_independent_output.genOrderIndependentOutput(" ", ["angry ","kind "], " ", modelLlama, tokenizerLlama, is_order_independent=True, max_new_tokens=2)

Modify llama model to accept 2D attention mask
Gen text with attention mask with shape torch.Size([1, 1, 7, 7])
Position IDs=tensor([[0, 1, 2, 3, 2, 3, 4]])
tensor([[[[1., 0., 0., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0., 0., 0.],
          [1., 1., 1., 0., 0., 0., 0.],
          [1., 1., 1., 1., 0., 0., 0.],
          [1., 1., 0., 0., 1., 0., 0.],
          [1., 1., 0., 0., 1., 1., 0.],
          [1., 1., 1., 1., 1., 1., 1.]]]])
4D attention mask torch.Size([1, 1, 7, 7])
tensor([[[[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38

In [4]:
g5Dup3,t5Dup3=gen_order_independent_output.genOrderIndependentOutput(" ", ["kind ","angry "], " ", modelLlama, tokenizerLlama, is_order_independent=True, max_new_tokens=2)

Modify llama model to accept 2D attention mask
Gen text with attention mask with shape torch.Size([1, 1, 7, 7])
Position IDs=tensor([[0, 1, 2, 3, 2, 3, 4]])
tensor([[[[1., 0., 0., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0., 0., 0.],
          [1., 1., 1., 0., 0., 0., 0.],
          [1., 1., 1., 1., 0., 0., 0.],
          [1., 1., 0., 0., 1., 0., 0.],
          [1., 1., 0., 0., 1., 1., 0.],
          [1., 1., 1., 1., 1., 1., 1.]]]])
2D attention mask shape=torch.Size([1, 8]),torch.float32
tensor([[[[1., 0., 0., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0., 0., 0.],
          [1., 1., 1., 0., 0., 0., 0.],
          [1., 1., 1., 1., 0., 0., 0.],
          [1., 1., 0., 0., 1., 0., 0.],
          [1., 1., 0., 0., 1., 1., 0.],
          [1., 1., 1., 1., 1., 1., 1.]]]])
After tensor([[1., 1., 1., 1., 1., 1., 1., 1.]])
Position IDs=tensor([[5]])
tensor([[1., 1., 1., 1., 1., 1., 1., 1.]])
1D attention mask shape=torch.Size([1, 9]) torch.float32


In [8]:
from attention_mask_editing import scores_diff
print(repr(t5Dup2),"split",repr(t5Dup3))
scores_diff(g5Dup2,g5Dup3, add_epsilon=True)


'    = ' split '   \n\n'


c:\Users\Katrina\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:2916: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


tensor(0.0012)

In [6]:
import transformers
transformers.__version__

'4.38.0.dev0'

In [3]:
g5Dup,t5Dup=gen_order_independent_output.genOrderIndependentOutput(" ", ["hi","hello"], " ", modelLlama, tokenizerLlama, is_order_independent=True, max_new_tokens=2)

Modify llama model to accept 2D attention mask
2D attention mask shape=torch.Size([1, 6]),torch.float32
tensor([[[[1., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 0., 0.],
          [1., 1., 0., 1., 0.],
          [1., 1., 1., 1., 1.]]]])
After tensor([[1., 1., 1., 1., 1., 1.]])
1D attention mask shape=torch.Size([1, 7]) torch.float32


In [ ]:
g7Dup,t7Dup=gen_order_independent_output.genOrderIndependentOutput(" ", ["hi","hello"], " ", modelLlama, tokenizerLlama, is_order_independent=False, max_new_tokens=2)

## Llama - effect of forward() call

In [15]:
import importlib
import modeling_llama_attention
importlib.reload(modeling_llama_attention)

<module 'modeling_llama_attention' from 'c:\\Users\\Katrina\\OneDrive - Harvard University\\Documents\\Harvard\\Research\\thesis\\triple_queries_order_dependence\\modeling_llama_attention.py'>

In [16]:
from attention_mask_editing import *
from modeling_gpt_attention_refactored import get_2D_attention_accepting_model_gpt
from modeling_llama_attention import get_2D_attention_accepting_model_llama
from transformers import GPT2LMHeadModel, LlamaForCausalLM
def get_2D_attention_accepting_model(model):
    if isinstance(model,GPT2LMHeadModel):
        return get_2D_attention_accepting_model_gpt(model)
    elif isinstance(model,LlamaForCausalLM):
        print(f"Modify llama model to accept 2D attention mask")
        return modeling_llama_attention.get_2D_attention_accepting_model_llama(model)
    else:
        raise ValueError(f"model_type={model.__class__} not recognized")
def genForward(prefix, parallel_substrings, suffix, model, tokenizer, max_new_tokens=10, is_order_independent=True, reverse_parallel_substrings_order=False, torch_device="cpu", parallel_position_ids=True):
    # Modify the given model to accept a 2D attention mask as input
    model = get_2D_attention_accepting_model(model)
    
    # Tokenize input text
    tokA=tokenizer(prefix, return_tensors='pt',add_special_tokens=True, return_token_type_ids=False).to(torch_device)
    tokD=tokenizer(suffix, return_tensors='pt',add_special_tokens=False, return_token_type_ids=False).to(torch_device)
    if reverse_parallel_substrings_order:
        parallel_substrings=parallel_substrings[::-1]
    tokParallel = tuple([tokenizer(input_text, return_tensors='pt', add_special_tokens=False).to(torch_device) for input_text in parallel_substrings])
    tokAll=gen_order_independent_output.get_tokenized_input_prompt(tokA,tokParallel,tokD)
    assert(len(tokA['attention_mask'][0]) + sum([len(tokOption['attention_mask'][0]) for tokOption in tokParallel]) + len(tokD['attention_mask'][0]) == len(tokAll['attention_mask'][0]))
    s=len(tokAll['input_ids'][0])
    inputTextLen=len(prefix)+sum([len(s) for s in parallel_substrings])+len(suffix)
    
    if not is_order_independent:
        # Run tests with no attention mask nor position_id intervention
        custom_position_ids, tokParallel, tokAll = get_position_ids_padded_n_options(tokA, tokParallel, tokD, tokenizer=tokenizer)
        s=len(tokAll['input_ids'][0])
        causal_mask = torch.tril(torch.ones((s, s), dtype=torch.int)).view(1, 1, s, s)
        position_ids=torch.arange(s).unsqueeze(0)
        if parallel_position_ids:
            position_ids = custom_position_ids
        generated=model(tokAll['input_ids'], attention_mask=causal_mask, position_ids=position_ids)
    else:
        # Pad all parallel substrings to the same length, then generate a 2D attention mask such that all substrings are processed in parallel
        position_ids, tokParallel, tokAll = get_position_ids_padded_n_options(tokA, tokParallel, tokD, tokenizer=tokenizer)
        if not parallel_position_ids:
            position_ids = torch.arange(len(tokAll['input_ids'][0])).unsqueeze(0)
        attention_mask_2d = get_attention_mask_2d_n_options(tokA, tokParallel, tokD, tokAll).to(torch.int32)
        print(f"Gen text with attention mask with shape {attention_mask_2d.shape}, position_ids {position_ids}")
        print(attention_mask_2d)
        generated=model(tokAll['input_ids'], attention_mask=attention_mask_2d, position_ids=position_ids)
    return generated

## Inspect ROPE relative positional embedding

In [17]:
forwardOI_1=genForward(" ", ["kind ","angry "], " ", modelLlama, tokenizerLlama, is_order_independent=True, max_new_tokens=2, parallel_position_ids=True)

Modify llama model to accept 2D attention mask
Override attention
Gen text with attention mask with shape torch.Size([1, 1, 7, 7]), position_ids tensor([[0, 1, 2, 3, 2, 3, 4]])
tensor([[[[1, 0, 0, 0, 0, 0, 0],
          [1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0],
          [1, 1, 0, 0, 1, 0, 0],
          [1, 1, 0, 0, 1, 1, 0],
          [1, 1, 1, 1, 1, 1, 1]]]], dtype=torch.int32)
Position IDs=tensor([[0, 1, 2, 3, 2, 3, 4]])
tensor([[[[1, 0, 0, 0, 0, 0, 0],
          [1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0],
          [1, 1, 0, 0, 1, 0, 0],
          [1, 1, 0, 0, 1, 1, 0],
          [1, 1, 1, 1, 1, 1, 1]]]], dtype=torch.int32)
4D attention mask torch.Size([1, 1, 7, 7])
tensor([[[[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
 

## Inspect effect of individual position id vs attention mask edits

In [68]:
forwardOD_1=genForward(" ", ["kind ","angry "], " ", modelLlama, tokenizerLlama, is_order_independent=False, max_new_tokens=2)

Modify llama model to accept 2D attention mask
Position IDs=tensor([[0, 1, 2, 3, 4, 5, 6]])
tensor([[[[1, 0, 0, 0, 0, 0, 0],
          [1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0],
          [1, 1, 1, 1, 1, 0, 0],
          [1, 1, 1, 1, 1, 1, 0],
          [1, 1, 1, 1, 1, 1, 1]]]], dtype=torch.int32)
4D attention mask torch.Size([1, 1, 7, 7])
tensor([[[[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.00

In [92]:
forwardOI_1=genForward(" ", ["kind ","angry "], " ", modelLlama, tokenizerLlama, is_order_independent=True, max_new_tokens=2, parallel_position_ids=True)

Modify llama model to accept 2D attention mask
Gen text with attention mask with shape torch.Size([1, 1, 7, 7]), position_ids tensor([[0, 1, 2, 3, 2, 3, 4]])
tensor([[[[1, 0, 0, 0, 0, 0, 0],
          [1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0],
          [1, 1, 0, 0, 1, 0, 0],
          [1, 1, 0, 0, 1, 1, 0],
          [1, 1, 1, 1, 1, 1, 1]]]], dtype=torch.int32)
Position IDs=tensor([[0, 1, 2, 3, 2, 3, 4]])
tensor([[[[1, 0, 0, 0, 0, 0, 0],
          [1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0],
          [1, 1, 0, 0, 1, 0, 0],
          [1, 1, 0, 0, 1, 1, 0],
          [1, 1, 1, 1, 1, 1, 1]]]], dtype=torch.int32)
4D attention mask torch.Size([1, 1, 7, 7])
tensor([[[[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+

In [93]:
forwardOI_2=genForward(" ", ["kind ","angry "], " ", modelLlama, tokenizerLlama, is_order_independent=True, max_new_tokens=2, reverse_parallel_substrings_order=True, parallel_position_ids=True)

Modify llama model to accept 2D attention mask
Gen text with attention mask with shape torch.Size([1, 1, 7, 7]), position_ids tensor([[0, 1, 2, 3, 2, 3, 4]])
tensor([[[[1, 0, 0, 0, 0, 0, 0],
          [1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0],
          [1, 1, 0, 0, 1, 0, 0],
          [1, 1, 0, 0, 1, 1, 0],
          [1, 1, 1, 1, 1, 1, 1]]]], dtype=torch.int32)
Position IDs=tensor([[0, 1, 2, 3, 2, 3, 4]])
tensor([[[[1, 0, 0, 0, 0, 0, 0],
          [1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0],
          [1, 1, 0, 0, 1, 0, 0],
          [1, 1, 0, 0, 1, 1, 0],
          [1, 1, 1, 1, 1, 1, 1]]]], dtype=torch.int32)
4D attention mask torch.Size([1, 1, 7, 7])
tensor([[[[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+

In [110]:
# NOTE: for order independent output, we would expect that torch.allclse(forwardOI_1.logits[0][-1],forwardOI_2.logits[0][-1]). That is, the output logits for the last token in sequence D should be the same
for i in range(7):
    print(torch.allclose(forwardOI_1.logits[0][i],forwardOI_2.logits[0][i]),forwardOI_1.logits[0][i].shape)

True torch.Size([32000])
True torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])


In [26]:
# Check output logits
print(torch.allclose(forwardOI_1.logits,forwardOI_2.logits))

False


In [94]:
# Logits have rows 3/4 swapped with rows 5/6, which is reasonable given the reverse order of the parallel substrings
print(forwardOI_1.logits,forwardOI_1.logits.shape)
print(forwardOI_2.logits,forwardOI_2.logits.shape)
for i in range(7):
    print(torch.allclose(forwardOI_1.logits[0][i],forwardOD_1.logits[0][i]),forwardOI_1.logits[0][i].shape)

tensor([[[ 0.2707,  0.0165,  0.2806,  ...,  1.4403,  2.0234,  0.7647],
         [-6.0510, -1.8250,  5.9284,  ...,  0.1648, -4.5255, -0.5254],
         [-4.7036, -9.6824, -0.7055,  ..., -3.1614, -4.1214, -4.4494],
         ...,
         [-5.9388, -4.7156,  2.1508,  ..., -3.5959, -6.0349, -5.1460],
         [-7.3868, -8.1209,  3.3819,  ..., -1.0943, -6.0555, -3.4025],
         [-8.6184, -7.9483,  1.8119,  ..., -3.5772, -6.2585, -5.3752]]],
       grad_fn=<UnsafeViewBackward0>) torch.Size([1, 7, 32000])
tensor([[[  0.2707,   0.0165,   0.2806,  ...,   1.4403,   2.0234,   0.7647],
         [ -6.0510,  -1.8250,   5.9284,  ...,   0.1648,  -4.5255,  -0.5254],
         [ -5.9388,  -4.7156,   2.1508,  ...,  -3.5959,  -6.0349,  -5.1460],
         ...,
         [ -4.7036,  -9.6824,  -0.7055,  ...,  -3.1614,  -4.1214,  -4.4494],
         [ -6.4330, -10.4261,   2.0405,  ...,  -1.2784,  -4.0565,  -1.6630],
         [ -8.6184,  -7.9483,   1.8119,  ...,  -3.5772,  -6.2585,  -5.3752]]],
       grad_fn=<

In [95]:
for i in range(7):
    print(torch.allclose(forwardOI_1.logits[0][i],forwardOI_2.logits[0][i]),forwardOI_1.logits[0][i].shape)

True torch.Size([32000])
True torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])


In [33]:
print(len(forwardOI_1["past_key_values"]))
forwardOI_1["past_key_values"][0][0].shape

32


torch.Size([1, 32, 7, 128])

In [60]:
modelGPT = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizerGPT = GPT2Tokenizer.from_pretrained('gpt2')
tokenizerGPT.pad_token_id = tokenizerGPT.eos_token_id

In [46]:
forwardOI_1_gpt=genForward(" ", ["kind ","angry "], " ", modelGPT, tokenizerGPT, is_order_independent=True, max_new_tokens=2)
forwardOI_2_gpt=genForward(" ", ["kind ","angry "], " ", modelGPT, tokenizerGPT, is_order_independent=True, max_new_tokens=2, reverse_parallel_substrings_order=True)
print(torch.allclose(forwardOI_1_gpt.past_key_values[0],forwardOI_2_gpt.past_key_values[0]))
print(forwardOI_1_gpt.past_key_values,len(forwardOI_1_gpt.past_key_values))
print(forwardOI_2_gpt.past_key_values,len(forwardOI_2_gpt.past_key_values))

Gen text with attention mask with shape torch.Size([1, 1, 8, 8])
tensor([[[[1., 0., 0., 0., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0., 0., 0., 0.],
          [1., 1., 1., 0., 0., 0., 0., 0.],
          [1., 1., 1., 0., 0., 0., 0., 0.],
          [1., 0., 0., 0., 1., 0., 0., 0.],
          [1., 0., 0., 0., 1., 1., 0., 0.],
          [1., 0., 0., 0., 1., 1., 1., 0.],
          [1., 1., 1., 0., 1., 1., 1., 1.]]]])
Gen text with attention mask with shape torch.Size([1, 1, 8, 8])
tensor([[[[1., 0., 0., 0., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0., 0., 0., 0.],
          [1., 1., 1., 0., 0., 0., 0., 0.],
          [1., 1., 1., 1., 0., 0., 0., 0.],
          [1., 0., 0., 0., 1., 0., 0., 0.],
          [1., 0., 0., 0., 1., 1., 0., 0.],
          [1., 0., 0., 0., 1., 1., 0., 0.],
          [1., 1., 1., 1., 1., 1., 0., 1.]]]])


TypeError: allclose(): argument 'input' (position 1) must be Tensor, not tuple

In [67]:
forwardOD_1_gpt=genForward(" ", ["kind ","angry "], " ", modelGPT, tokenizerGPT, is_order_independent=False, max_new_tokens=2)
print(torch.allclose(forwardOI_1_gpt.logits,forwardOD_1_gpt.logits))
# NOTE: logits should be different for rows of tokens in sequence D
for i in range(8):
    print(torch.allclose(forwardOI_1_gpt.logits[0][i],forwardOD_1_gpt.logits[0][i]),forwardOI_1_gpt.logits[0][i].shape)
print(forwardOI_1_gpt.logits,forwardOI_1_gpt.logits.shape)
print(forwardOD_1_gpt.logits,forwardOD_1_gpt.logits.shape)

False
True torch.Size([50257]) torch.Size([50257])
True torch.Size([50257]) torch.Size([50257])
True torch.Size([50257]) torch.Size([50257])
False torch.Size([50257]) torch.Size([50257])
False torch.Size([50257]) torch.Size([50257])
False torch.Size([50257]) torch.Size([50257])
False torch.Size([50257]) torch.Size([50257])
False torch.Size([50257]) torch.Size([50257])
tensor([[[-31.3032, -30.2778, -32.2991,  ..., -40.4723, -39.6234, -30.6677],
         [-60.1542, -60.0518, -64.5230,  ..., -68.5682, -69.0504, -62.2265],
         [-46.4999, -49.3436, -51.6780,  ..., -57.4955, -56.3297, -50.5656],
         ...,
         [-74.7438, -72.2002, -77.9424,  ..., -81.5106, -80.7995, -75.5365],
         [-51.8716, -52.6931, -54.2863,  ..., -60.9329, -58.7848, -54.7333],
         [-50.6570, -51.7063, -52.8174,  ..., -60.3168, -57.9888, -52.1964]]],
       grad_fn=<UnsafeViewBackward0>) torch.Size([1, 8, 50257])
tensor([[[-31.3032, -30.2778, -32.2991,  ..., -40.4723, -39.6234, -30.6677],
         [

In [70]:
# forward(ABCD).logits vs forward(ACBD).logits for GPT - sequence D logits are the same! 
for i in range(8):
    print(torch.allclose(forwardOI_1_gpt.logits[0][i],forwardOI_2_gpt.logits[0][i]),forwardOI_1_gpt.logits[0][i].shape)

True torch.Size([50257])
False torch.Size([50257])
False torch.Size([50257])
False torch.Size([50257])
False torch.Size([50257])
False torch.Size([50257])
False torch.Size([50257])
True torch.Size([50257])


In [112]:
# forward(ABCD).logits vs forward(ACBD).logits for Llama - sequence D logits are different!
# logits are swapped for positions 2/4 and 3/5 due to reversing the order of the parallel substrings, but logits are otherwise the same. This indicates that C tokens don't attend to B tokens, as desired??
print(torch.allclose(forwardOI_1.logits[0][2],forwardOI_2.logits[0][4]))
print(torch.allclose(forwardOI_1.logits[0][3],forwardOI_2.logits[0][5])) 
for i in range(7):
    print(torch.allclose(forwardOI_1.logits[0][i],forwardOI_2.logits[0][i]),forwardOI_1.logits[0][i].shape)

True
True
True torch.Size([32000])
True torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])


In [74]:
# forward(ABCD).logits order independent vs forward(ABCD).logits order dependent for Llama - sequence D logits are different!
for i in range(7):
    print(torch.allclose(forwardOI_1.logits[0][i],forwardOD_1.logits[0][i]),forwardOI_1.logits[0][i].shape)

True torch.Size([32000])
True torch.Size([32000])
True torch.Size([32000])
True torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])


In [103]:
# forward(ABCD).logits parallel attention mask but default position ids vs forward(ABCD).logits parallel attention mask and parallel position ids for Llama
forwardOI_1_default_position=genForward(" ", ["kind ","angry "], " ", modelLlama, tokenizerLlama, is_order_independent=True, max_new_tokens=2, reverse_parallel_substrings_order=False,parallel_position_ids=False)

Modify llama model to accept 2D attention mask
Gen text with attention mask with shape torch.Size([1, 1, 7, 7]), position_ids tensor([[0, 1, 2, 3, 4, 5, 6]])
tensor([[[[1, 0, 0, 0, 0, 0, 0],
          [1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0],
          [1, 1, 0, 0, 1, 0, 0],
          [1, 1, 0, 0, 1, 1, 0],
          [1, 1, 1, 1, 1, 1, 1]]]], dtype=torch.int32)
Position IDs=tensor([[0, 1, 2, 3, 4, 5, 6]])
tensor([[[[1, 0, 0, 0, 0, 0, 0],
          [1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0],
          [1, 1, 0, 0, 1, 0, 0],
          [1, 1, 0, 0, 1, 1, 0],
          [1, 1, 1, 1, 1, 1, 1]]]], dtype=torch.int32)
4D attention mask torch.Size([1, 1, 7, 7])
tensor([[[[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+

In [102]:
# Basically, does inputting custom position_ids change the output logits, while keeping default attention mask constant?
# Only C,D logits are different, which is expected because the position_ids are different
forwardOD_1_custom_position = genForward(" ", ["kind ","angry "], " ", modelLlama, tokenizerLlama, is_order_independent=False, max_new_tokens=2, reverse_parallel_substrings_order=False,parallel_position_ids=True)
for i in range(7):
    print(torch.allclose(forwardOD_1.logits[0][i],forwardOD_1_custom_position.logits[0][i]),forwardOI_1.logits[0][i].shape)

Modify llama model to accept 2D attention mask
Position IDs=tensor([[0, 1, 2, 3, 2, 3, 4]])
tensor([[[[1, 0, 0, 0, 0, 0, 0],
          [1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0],
          [1, 1, 1, 1, 1, 0, 0],
          [1, 1, 1, 1, 1, 1, 0],
          [1, 1, 1, 1, 1, 1, 1]]]], dtype=torch.int32)
4D attention mask torch.Size([1, 1, 7, 7])
tensor([[[[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.00

In [105]:
# forward(ABCD).logits order independent vs forward(ABCD).logits order dependent for Llama - sequence D logits are ???
# Basically, does inputting custom position ids change the output logits? It should, for C/D tokens only, right?
# We'd expect that inputting custom position ids tensor([[0, 1, 2, 3, 2, 3, 4]] instead of default tensor([[0, 1, 2, 3, 4, 5, 6]] would change output logits for C and D
# Instead, inputting custom position ids changes the output logits for B,C, and D tokens. Why do custom position ids affect the output logits for B tokens?
for i in range(7):
    print(torch.allclose(forwardOI_1.logits[0][i],forwardOI_1_default_position.logits[0][i]),forwardOI_1.logits[0][i].shape)

True torch.Size([32000])
True torch.Size([32000])
True torch.Size([32000])
True torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])


In [109]:
# Does inputting a custom attention mask change the output logits for C/D tokens only? Default position ids for both
# Changing the attention mask changes the output logits for tokens C,D only as expected
for i in range(7):
    print(torch.allclose(forwardOD_1.logits[0][i],forwardOI_1_default_position.logits[0][i]),forwardOI_1.logits[0][i].shape)

True torch.Size([32000])
True torch.Size([32000])
True torch.Size([32000])
True torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])


In [108]:
# Basically, intervening on both attention mask and position ids affects only the C,D output logits
for i in range(7):
    print(torch.allclose(forwardOD_1.logits[0][i],forwardOI_1.logits[0][i]),forwardOI_1.logits[0][i].shape)

True torch.Size([32000])
True torch.Size([32000])
True torch.Size([32000])
True torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])
False torch.Size([32000])


In [51]:
print(len(forwardOI_1_gpt.past_key_values))
print(len(forwardOI_1_gpt.past_key_values[0]))
print(forwardOI_1_gpt.past_key_values[0][0].shape)

12
2
torch.Size([1, 12, 8, 64])


In [54]:
for i in range(len(forwardOI_1_gpt.past_key_values)):
    for j in range(len(forwardOI_1_gpt.past_key_values[i])):
        print(torch.allclose(forwardOI_1_gpt.past_key_values[i][j],forwardOI_2_gpt.past_key_values[i][j]))

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
